In [291]:
import sklearn
sklearn.__version__
import pandas as pd  

import time

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')


In [292]:
sample = pd.read_csv('train.csv')
testdatax= pd.read_csv('test.csv')

In [293]:
# sample.drop(['RecordID'], axis=1, inplace=True)
# testdatax.drop(['RecordID'], axis=1, inplace=True)
# sample.drop(['hospital_id'], axis=1, inplace=True)
# testdatax.drop(['hospital_id'], axis=1, inplace=True)
# sample.drop(['icu_id'], axis=1, inplace=True)
# testdatax.drop(['icu_id'], axis=1, inplace=True)

In [294]:
text_to_find = 'noninvasive'
columns_to_drop = [col for col in sample.columns if text_to_find in col]
sample.drop(columns=columns_to_drop, inplace=True)
columns_to_drop = [col for col in testdatax.columns if text_to_find in col]
testdatax.drop(columns=columns_to_drop, inplace=True)

In [295]:
ob_col= sample.select_dtypes(include=['object']).columns
not_ob_col= sample.select_dtypes(exclude=['object']).columns
not_ob_col_age= sample.select_dtypes(exclude=['object']).columns

not_ob_col.delete([-1])
ob_col

Index(['ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type',
       'apache_3j_bodysystem', 'apache_2_bodysystem'],
      dtype='object')

In [296]:
# for i in ob_col:
#     mode_val= sample[i].mode()[0]
#     sample[i].fillna(mode_val,inplace=True)


# for i in ob_col:
#     mode_val= testdatax[i].mode()[0]
#     testdatax[i].fillna(mode_val,inplace=True)    

In [297]:
scaler = RobustScaler( )
for i in not_ob_col:
    sample[i]= scaler.fit_transform(sample[i].values.reshape(-1, 1))

not_ob_col_test= testdatax.select_dtypes(exclude=['object']).columns
not_ob_col_test.delete(0)
for i in not_ob_col_test:
    testdatax[i]= scaler.fit_transform(testdatax[i].values.reshape(-1, 1))

In [298]:
# for i in not_ob_col_age:
#     mean_val=sample[i].median()
#     sample[i].fillna(mean_val,inplace=True)

# not_ob_col_age_test= testdatax.select_dtypes(exclude=['object']).columns
# for i in not_ob_col_age_test:
#     mean_val=testdatax[i].median()
#     testdatax[i].fillna(mean_val,inplace=True)


In [299]:
# label_encoders = {}
# for col in ob_col:
#     le = LabelEncoder()
#     sample[col] = le.fit_transform(sample[col])
#     label_encoders[col] = le


# label_encoders1 = {}
# for col in ob_col:
#     le = LabelEncoder()
#     testdatax[col] = le.fit_transform(testdatax[col])
#     label_encoders1[col] = le

In [300]:
sample.isnull().sum()

RecordID                            0
hospital_id                         0
icu_id                              0
ethnicity                         576
gender                              4
icu_admit_source                   47
icu_stay_type                       0
icu_type                            0
apache_3j_bodysystem              276
apache_2_bodysystem               276
age                              1801
elective_surgery                    0
pre_icu_los_days                    0
apache_2_diagnosis                276
apache_3j_diagnosis                64
apache_post_operative               0
gcs_eyes_apache                   405
gcs_motor_apache                  405
gcs_unable_apache                  42
gcs_verbal_apache                 405
heart_rate_apache                   2
intubated_apache                    0
resprate_apache                    62
temp_apache                      1249
ventilated_apache                   0
d1_diasbp_min                       0
d1_heartrate

In [301]:
df_onehot_test = pd.get_dummies(testdatax)
df_onehot_sample = pd.get_dummies(sample)

df_onehot_test.dtypes 


RecordID                                   float64
hospital_id                                float64
icu_id                                     float64
age                                        float64
elective_surgery                           float64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary       bool
apache_2_bodysystem_Respiratory               bool
apache_2_bodysystem_Trauma                    bool
apache_2_bodysystem_Undefined Diagnoses       bool
apache_2_bodysystem_Undefined diagnoses       bool
Length: 87, dtype: object

In [302]:
df_onehot_sample.isnull().sum()

RecordID                                      0
hospital_id                                   0
icu_id                                        0
age                                        1801
elective_surgery                              0
                                           ... 
apache_2_bodysystem_Renal/Genitourinary       0
apache_2_bodysystem_Respiratory               0
apache_2_bodysystem_Trauma                    0
apache_2_bodysystem_Undefined Diagnoses       0
apache_2_bodysystem_Undefined diagnoses       0
Length: 88, dtype: int64

In [303]:
# knimp= KNNImputer(n_neighbors=10001,weights='distance')

# knimp.fit_transform(df_onehot_sample.values.reshape(-1, 1))
# knimp.fit_transform(df_onehot_test.values.reshape(-1, 1)) 

In [304]:
# X = sample.iloc[:,sample.columns != 'hospital_death']
# y = sample['hospital_death']

In [305]:
# lgb_model = lgb.LGBMClassifier(max_depth=3, n_estimators=300, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8)
# sfs = SequentialFeatureSelector(lgb_model, direction='forward',n_features_to_select=15, scoring='roc_auc', n_jobs=-1,)
# X = df_onehot_sample.loc[:, df_onehot_sample.columns != 'hospital_death']
# y = df_onehot_sample[['hospital_death']]
# sfs.fit(X, y)
# print(sfs.get_feature_names_out())

In [306]:
X = df_onehot_sample.iloc[:,df_onehot_sample.columns != 'hospital_death']
y = df_onehot_sample['hospital_death']

In [307]:
def fit_model(model, model_name):
    model.fit(trainX,trainy)
    md_probs = model.predict_proba(test_2X)
    md_probs = md_probs[:,1]
    md_auc = roc_auc_score(testy, md_probs)
    print(model_name, " : ", md_auc)
    md_fpr, md_tpr, _ = roc_curve(testy, md_probs)
    # pyplot.plot(md_fpr, md_tpr, marker='.', label=model_name)
    #return (md_fpr, md_tpr)

In [319]:
trainX, test_2X, trainy, testy = train_test_split(X, y, test_size=0.3,random_state=2)



Bg = BaggingClassifier(base_estimator=XGBClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8,colsample_bytree=0.7,boosting_type='gbdt',num_leaves=10),n_estimators=100,n_jobs=-1,warm_start=True) 
fit_model(Bg, "Voting Classifier")



/Users/irtiza/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:160: UserWarning: [23:43:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "boosting_type", "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/irtiza/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:160: UserWarning: [23:43:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "boosting_type", "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/irtiza/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:160: UserWarning: [23:43:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "boosting_type", "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/irtiza/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:160: UserWarning: [23:43:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "boosting_type", "num_leav

Voting Classifier  :  0.8886920075686642


In [ ]:
Voting Classifier  :  0.8886905850157724
Voting Classifier  :  0.8885898113689176

In [ ]:

# xgb_model = xgb.XGBClassifier(max_depth=15, n_estimators=300, learning_rate=0.1,sampling_method="gradient_based")
# param_grid = {'tree_method': ['auto','exact','approx','hist']}
# grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='roc_auc')
# grid_search.fit(X, y)
# ab = AdaBoostClassifier(n_estimators=300)
#xgb_model = xgb.XGBClassifier(max_depth=4, n_estimators=100, learning_rate=0.1,objective='reg:logistic',min_child_weight=30,col_sample_bytree=0.8,subsample=0.9,reg_lambda=0.8,n_jobs=-1,reg_alpha=0.9)
#cb = CatBoostClassifier(iterations=num_of_models, depth=depth_level, learning_rate=0.1, loss_function='Logloss', verbose=False)
# lgb_model = lgb.LGBMClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8,colsample_bytree=0.8,boosting_type='gbdt',num_leaves=10)
# bg_c = BaggingClassifier(base_estimator=lgb_model, n_estimators=100, n_jobs=-1)
# vc = VotingClassifier(estimators=[('xgB', bg1), ('cb', bg2) , ('lgbm' ,bg3)], voting='soft',n_jobs=-1,weights=[2,1,1,1])

In [320]:
from catboost import CatBoostClassifier



# estimators = [('xgb',XGBClassifier(n_estimators=349,learning_rate=0.1, max_depth=3,subsample = 0.8,min_child_weight=3)),
#                 ('knn', BaggingClassifier(base_estimator=lgb.LGBMClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8,colsample_bytree=0.8,boosting_type='gbdt',num_leaves=10),n_estimators=100,n_jobs=-1)),
#                 ('nb',CatBoostClassifier(n_estimators=373, learning_rate=0.1,max_depth=3, subsample = 0.8) )]
              
# st_clf = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier(base_estimator=lgb.LGBMClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8,colsample_bytree=0.8,boosting_type='gbdt',num_leaves=10),n_estimators=100,n_jobs=-1))

# st_clf.fit(X,y)
# md_probs = st_clf.predict_proba(df_onehot_test)
# md_probs = md_probs[:,1]
# md_probs

# max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,
# reg_alpha=0.9,subsample=0.8,colsample_bytree=0.7,boosting_type='gbdt',num_leaves=10),
# n_estimators=100,n_jobs=-1,warm_start=True

# lgbm_params 
xgb_params= {
    'learning_rate': 0.1,
    'max_depth': 3,
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'reg_alpha': 0.9,
    'reg_lambda': 0.8,
    'n_jobs': -1,
    'num_leaves': 10
}

# Create a LightGBMClassifier as the base estimator
# base_lgbm_model = lgb.LGBMClassifier(n_estimators=500, random_state=42, **lgbm_params)

# Create a BaggingClassifier with LightGBM as the base estimator and warm start enabled
# Bg = BaggingClassifier(base_estimator=base_lgbm_model, n_estimators=215, random_state=42, warm_start=True)

# Create an XGBoostClassifier as the base estimator
base_xgb_model = XGBClassifier(n_estimators=500, random_state=42, **xgb_params)

# Create a BaggingClassifier with XGBoost as the base estimator and warm start enabled
Bg = BaggingClassifier(base_estimator=base_xgb_model, n_estimators=300, random_state=42, warm_start=True)




# Fit the BaggingClassifier

#Bg = BaggingClassifier(base_estimator=lgb.LGBMClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,n_jobs=-1,reg_lambda=0.8,reg_alpha=0.9,subsample=0.8,colsample_bytree=0.9,boosting_type='gbdt',num_leaves=10),n_estimators=200,n_jobs=-1,) 
Bg.fit(X,y)
md_probs = Bg.predict_proba(df_onehot_test)
md_probs = md_probs[:,1]



In [321]:
df = pd.DataFrame(md_probs)
custom_starting_index = 50001
df.insert(0, 'RecordID', range(custom_starting_index, custom_starting_index + len(df)))

custom_column_names = ['RecordID', 'hospital_death']

csv_file_path = ' predict_lgbm_bagging.csv'

df.to_csv(csv_file_path, index=False, header=custom_column_names)